In [11]:
import pandas as pd # Need Pandas, yo!

## Combing Multiple CSV Files

In [12]:
### Combining all the NYSE .csv files
import glob
    # module to used to retrieve files/pathnames files, https://docs.python.org/3/library/glob.html

source_files = sorted(glob.glob('data/NYSE/*.csv'))
  # .glob('yourpath\*.csv') generates a list containing the file names
  # sorted() arranges the list in alphabetical order

### Iterate through source_files to combine into one df
dataframe_list = [] # generating a list containing the df from each file
for file in source_files:
    df = pd.read_csv(file)
    dataframe_list.append(df)

# Concatenate the list, ie combine into one 
df_NYSE = pd.concat(dataframe_list)

In [13]:
df_NYSE

,Symbol,Date,Open,High,Low,Close,Volume
0,A,02 Jan 2023,149.6500,149.6500,149.6500,149.6500,0
1,AA,02 Jan 2023,45.4700,45.4700,45.4700,45.4700,0
2,AAC,02 Jan 2023,10.0700,10.0700,10.0700,10.0700,0
3,AAC.U,02 Jan 2023,10.1700,10.1700,10.1700,10.1700,0
4,AAC.W,02 Jan 2023,0.4663,0.4663,0.4663,0.4663,0
...,...,...,...,...,...,...,...
2946,ZTR,03 Nov 2023,5.0000,5.0979,5.0000,5.0800,328500
2947,ZTS,03 Nov 2023,164.2500,164.7200,159.3700,162.2300,1974800
2948,ZUO,03 Nov 2023,7.6500,7.8000,7.4800,7.7500,512100
2949,ZVIA,03 Nov 2023,2.0000,2.2661,1.9687,2.0700,247900


In [14]:
# df_NYSE[df_NYSE["Symbol"] == "VOO"] 
    # This seaches for a ticker and returns the rows
    # VOO does not exist in NYSE

In [15]:
# Exports df_NYSE to a csv file
# df_NYSE.to_csv("df_NYSE.csv", index=False)

### Data Clean

#### Ensure Stocks traded over the complete time period

In [16]:
### Ensure each stock has complete data
stock_instances = df_NYSE['Symbol'].value_counts() # returns the number of instances for each stock
stock_instances

Symbol
A        217
NOG      217
NMT      217
NMZ      217
NNI      217
        ... 
K.P        2
NLO.P      2
NLOP       2
TRAK       2
KCG.U      1
Name: count, Length: 3153, dtype: int64

In [17]:
filtered = stock_instances[stock_instances<217].index # returning stocks that have less than the full 217 data points
    # .index is syntax and returns the index only. Otherwise the series would the index and values
    # .tolist() could be added to the end of the line to return a list versus keeping it as a series
filtered

Index(['QBT.W', 'ATC-D', 'GLP-A', 'COD-B', 'OPP-B', 'GAB-K', 'TWN', 'WFC-L',
       'CNO-A', 'DLN-B',
       ...
       'VST.P', 'DHR.P', 'KLG.P', 'LAA.P', 'LAC.P', 'K.P', 'NLO.P', 'NLOP',
       'TRAK', 'KCG.U'],
      dtype='object', name='Symbol', length=578)

In [18]:
df_NYSE = df_NYSE[~df_NYSE['Symbol'].isin(filtered)]
    # df['symbol'].isin(stocks_to_remove) returns a boolean Series: True for rows you want to remove
    # ~ flips it: True becomes False, and vice versa
    # df[...] then keeps only the rows not in stocks_to_remove

df_NYSE

,Symbol,Date,Open,High,Low,Close,Volume
0,A,02 Jan 2023,149.65,149.6500,149.6500,149.65,0
1,AA,02 Jan 2023,45.47,45.4700,45.4700,45.47,0
2,AAC,02 Jan 2023,10.07,10.0700,10.0700,10.07,0
7,AAIC,02 Jan 2023,2.93,2.9300,2.9300,2.93,0
11,AAN,02 Jan 2023,11.95,11.9500,11.9500,11.95,0
...,...,...,...,...,...,...,...
2946,ZTR,03 Nov 2023,5.00,5.0979,5.0000,5.08,328500
2947,ZTS,03 Nov 2023,164.25,164.7200,159.3700,162.23,1974800
2948,ZUO,03 Nov 2023,7.65,7.8000,7.4800,7.75,512100
2949,ZVIA,03 Nov 2023,2.00,2.2661,1.9687,2.07,247900


In [19]:
### Test to ensure stocks were removed
# df_NYSE[df_NYSE['Symbol'] == 'A'] 

#### Improvement Ideas
-remove the days where there is no trading. However, this may impact calculate of percent gain/loss because the days may not be next to each other  
-Have program auto-determine stocks with complete data and then remove the ones that are not

# Outline
1. Start with one stock and get the data set aligned with VOO.
 -the dates need to aline
 -calculate the percent change

4. run correlation

5. Then move to iterate through complete stock exchange to find stocks that correlate

## Acheive Correlation with One Stock

Determine stocks that have a positive correlation with VOO to predict price increases
-Based on percent change from the previous day

Then see if when the group of stocks has a positive gain does that translate into a postive gain the next day for VOO

### Selecting a stock from NYSE

In [20]:
### Select a stock from NYSE dataset
# Berkshire Hathaway
df_BRK = df_NYSE[df_NYSE['Symbol'] == 'BRK.A'].copy()
    # using .copy() to make df_BRK a new df and not a view of df_NYSE
    # see ChatGTP chat, towards the end https://chatgpt.com/share/6823e473-f190-8005-a76d-adb3a633163f
df_BRK

,Symbol,Date,Open,High,Low,Close,Volume
461,BRK.A,02 Jan 2023,468711.0,468711.0,468711.0,468711.0,0
461,BRK.A,03 Jan 2023,472950.0,473990.0,465019.8,469525.0,5200
461,BRK.A,04 Jan 2023,477479.0,479897.1,470151.0,476259.9,3700
461,BRK.A,05 Jan 2023,481229.0,481229.0,469227.5,473660.0,3400
461,BRK.A,06 Jan 2023,475989.8,484150.6,473300.0,481955.0,3800
...,...,...,...,...,...,...,...
452,BRK.A,30 Oct 2023,505702.5,513700.0,504050.1,513065.0,8800
453,BRK.A,31 Oct 2023,513630.2,518780.0,512036.3,517825.0,6600
451,BRK.A,01 Nov 2023,517662.0,538000.0,515921.3,523749.0,7400
451,BRK.A,02 Nov 2023,527362.0,530800.0,523701.0,530340.0,7100


### Creating VOO dataframe

In [21]:
### Reading the VOO dataset
df_VOO = pd.read_csv('Data/VOO/VOO_20100909_20231227.csv')

# Quick look at the dataset
df_VOO

,Symbol,Date,Open,High,Low,Close,Volume
0,VOO,09 Sep 2010,102.50,102.500,101.1400,101.32,53000
1,VOO,10 Sep 2010,101.68,101.860,101.3000,101.78,17200
2,VOO,13 Sep 2010,102.96,103.140,102.5000,103.06,67500
3,VOO,14 Sep 2010,102.84,103.480,102.3800,103.04,118800
4,VOO,15 Sep 2010,102.62,103.380,102.4000,103.30,18500
...,...,...,...,...,...,...,...
3460,VOO,20 Dec 2023,435.51,437.305,429.9000,430.09,5783200
3461,VOO,21 Dec 2023,433.13,434.640,430.8552,434.28,4621600
3462,VOO,22 Dec 2023,435.46,436.838,433.4800,435.29,4037800
3463,VOO,26 Dec 2023,435.67,437.920,435.5800,437.10,3835800


In [22]:
# df_VOO.describe()

### Adding percent change to both dfs

#### Percent change to the stock df

In [23]:
### Add column to df_BRK for the percent change

# Test to understand .iloc
print(df_BRK.iloc[0,5]) # syntax [row,column]

# Loop to calculate the percent of change
i = 0
percent_gain = []
while i < len(df_BRK['Close'])-1: # iterating over 'close' column, have to go -1 the length otherwise the loop goes out of 'bounds'
  percent_gain.append((df_BRK.iloc[i+1,5]-df_BRK.iloc[i,5])/df_BRK.iloc[i,5]) # percent change formula (x2-x1)/x1
  i+=1

# adding 0 at the begining of the list
# recall percent change is not possible for the first date in the df
percent_gain.insert(0,0) # syntax .insert(index,value) / lets you insert a new item before the given position, shifting all later items one slot to the right.
print(len(percent_gain))

# Add the percent_gain list to the df
df_BRK['Gain'] = percent_gain
df_BRK.index


468711.0
217


Index([461, 461, 461, 461, 461, 461, 461, 461, 461, 461,
       ...
       452, 460, 459, 454, 459, 452, 453, 451, 451, 449],
      dtype='int64', length=217)

#### Percent change to VOO df

In [24]:
### Add column to df_VOO for the percent change

# Test to understand .iloc
print(df_VOO.iloc[0,5]) # syntax [row,column]

# Loop to calculate the percent of change
i = 0
percent_gain = []
while i < len(df_VOO['Close'])-1: # iterating over 'close' column, have to go -1 the length otherwise the loop goes out of 'bounds'
  percent_gain.append((df_VOO.iloc[i+1,5]-df_VOO.iloc[i,5])/df_VOO.iloc[i,5]) # percent change formula (x2-x1)/x1
  i+=1

# adding 0 at the begining of the list
# recall percent change is not possible for the first date in the df
percent_gain.insert(0,0)
print(len(percent_gain))

# Add the percent_gain list to the df
df_VOO['Gain'] = percent_gain
df_VOO


101.32
3465


,Symbol,Date,Open,High,Low,Close,Volume,Gain
0,VOO,09 Sep 2010,102.50,102.500,101.1400,101.32,53000,0.000000
1,VOO,10 Sep 2010,101.68,101.860,101.3000,101.78,17200,0.004540
2,VOO,13 Sep 2010,102.96,103.140,102.5000,103.06,67500,0.012576
3,VOO,14 Sep 2010,102.84,103.480,102.3800,103.04,118800,-0.000194
4,VOO,15 Sep 2010,102.62,103.380,102.4000,103.30,18500,0.002523
...,...,...,...,...,...,...,...,...
3460,VOO,20 Dec 2023,435.51,437.305,429.9000,430.09,5783200,-0.018485
3461,VOO,21 Dec 2023,433.13,434.640,430.8552,434.28,4621600,0.009742
3462,VOO,22 Dec 2023,435.46,436.838,433.4800,435.29,4037800,0.002326
3463,VOO,26 Dec 2023,435.67,437.920,435.5800,437.10,3835800,0.004158


### Run correlation on percent change

In [25]:
### Aligning dates of the DFs
'''
-Use the data range of the stock to select the data range of VOO
-Recall first date does not have a percent change
-Intent is to have a leading indicator for VOO, so
'''

df_BRK_PC = df_BRK[['Date','Gain']].copy()
    # .copy() to create an independant df
df_VOO_PC = df_VOO[['Date','Gain']].copy()

## Merging the two dataframes
# already truncates VOO to the date range of the stock
result_df = pd.merge(df_BRK_PC, df_VOO_PC, on='Date', how='inner')
'''
on : Use the column named "Date" in both DataFrames as the key to align rows. Only rows where df_BRK_PC.Date == df_VOO_PC.Date will match.
how : The join type. An inner join keeps only the intersection of keys—that is, dates that appear in both DataFrames. Any date missing from one side is dropped.
'''
result_df



,Date,Gain_x,Gain_y
0,02 Jan 2023,0.000000,0.000000
1,03 Jan 2023,0.001737,-0.003842
2,04 Jan 2023,0.014344,0.007200
3,05 Jan 2023,-0.005459,-0.010922
4,06 Jan 2023,0.017513,0.022744
...,...,...,...
212,30 Oct 2023,0.019250,0.012032
213,31 Oct 2023,0.009278,0.006049
214,01 Nov 2023,0.011440,0.011011
215,02 Nov 2023,0.012584,0.019001


In [26]:
### Correlation of the stock to VOO, straight (no time shift)
result_df['Gain_x'].corr(result_df['Gain_y'])

0.6409548495225923

In [27]:
### Shift VOO
'''
Need to shift VOO pc down one. The intent is to have a leading indicator for VOO.
So performance on the stock the day before predicts the next day on VOO
'''
#remove date column
df1 = result_df.drop(['Date'], axis=1)

# shifting the stock down one to align the stock to next day's VOO value
df1['Gain_x'] = df1.Gain_x.shift(1,fill_value=0) # Shifting the stock
  # resource https://www.tutorialspoint.com/how-to-shift-a-column-in-a-pandas-dataframe
  # resource https://towardsdatascience.com/all-the-pandas-shift-you-should-know-for-data-analysis-791c1692b5e

# Removing the first two rows and the last
df2 = df1.drop([0,1,len(df1['Gain_y'])-1])
df2

,Gain_x,Gain_y
2,0.001737,0.007200
3,0.014344,-0.010922
4,-0.005459,0.022744
5,0.017513,-0.000729
6,-0.007408,0.007128
...,...,...
211,-0.004440,-0.004433
212,-0.013232,0.012032
213,0.019250,0.006049
214,0.009278,0.011011


In [28]:
### Find correlation
df2.corr(method='pearson')

,Gain_x,Gain_y
Gain_x,1.000000,0.022988
Gain_y,0.022988,1.000000


In [29]:
df2['Gain_x'].corr(df2['Gain_y'], method = 'pearson')

0.022987945745582314

## Iterate through all the stocks to generate correlations

In [30]:
df_NYSE['Symbol'].nunique() # Returns an integer: the count of unique values
    # .unique() Returns an array of all distinct values in the column.

2575

In [31]:
df_NYSE['Symbol'].unique()

array(['A', 'AA', 'AAC', ..., 'ZUO', 'ZVIA', 'ZWS'], dtype=object)

In [32]:
'''
.shape returns a tuple: (number_of_rows, number_of_columns)
So df.shape[0] gives you the row count
Or df.shape[1] gives you the column count

an alternative would be to use len(df_NYSE)
'''
df_NYSE.shape[0]

558775

In [33]:
### Loop
unique_stocks = df_NYSE['Symbol'].unique()

for ticker in unique_stocks:
    df_ticker = df_ticker = df_NYSE[df_NYSE['Symbol'] == ticker].copy()
    print(df_ticker.head(1))

  Symbol         Date    Open    High     Low   Close  Volume
0      A  02 Jan 2023  149.65  149.65  149.65  149.65       0
  Symbol         Date   Open   High    Low  Close  Volume
1     AA  02 Jan 2023  45.47  45.47  45.47  45.47       0
  Symbol         Date   Open   High    Low  Close  Volume
2    AAC  02 Jan 2023  10.07  10.07  10.07  10.07       0
  Symbol         Date  Open  High   Low  Close  Volume
7   AAIC  02 Jan 2023  2.93  2.93  2.93   2.93       0
   Symbol         Date   Open   High    Low  Close  Volume
11    AAN  02 Jan 2023  11.95  11.95  11.95  11.95       0
   Symbol         Date    Open    High     Low   Close  Volume
12    AAP  02 Jan 2023  147.03  147.03  147.03  147.03       0
   Symbol         Date  Open  High   Low  Close  Volume
13    AAT  02 Jan 2023  26.5  26.5  26.5   26.5       0
   Symbol         Date   Open   High    Low  Close  Volume
14     AB  02 Jan 2023  34.37  34.37  34.37  34.37       0
   Symbol         Date    Open    High     Low   Close  Volu

KeyboardInterrupt: 